In [1]:
import traceback
from collections import Counter
from pathlib import Path
from typing import Union

import anvil
from nbt.nbt import TAG

In [2]:
gtnh_instance_path = Path('~/.local/share/multimc/instances/GT_New_Horizons_2.6.1_Java_8/').expanduser()
mc_world_path = Path('.minecraft/saves/GotG seed')
region_file_path = Path('DIM1/region/r.0.0.mca')
full_path = gtnh_instance_path / mc_world_path / region_file_path

region: anvil.Region = anvil.Region.from_file(str(full_path))

print(region)
print(dir(region))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', 'chunk_data', 'chunk_location', 'data', 'from_file', 'get_chunk', 'header_offset']


In [3]:
# Note: 1.7.10 doesn't have a DataVersion,
#   but passing 100 fixes it since it only checks for self.version < _VERSION_17w47a:

# Overwrite get_chunk classmethod (can't pass in NBT in the original implementation)
def from_region_override(cls, region: Union[str, anvil.Region], chunk_x: int, chunk_z: int):
    """
    Creates a new chunk from region and the chunk's X and Z

    Parameters
    ----------
    region
        Either a :class:`anvil.Region` or a region file name (like ``r.0.0.mca``)

    Raises
    ----------
    anvil.ChunkNotFound
        If a chunk is outside this region or hasn't been generated yet
    """
    if isinstance(region, str):
        region = anvil.Region.from_file(region)
    nbt_data = region.chunk_data(chunk_x, chunk_z)
    if nbt_data is None:
        raise anvil.errors.ChunkNotFound(f'Could not find chunk ({chunk_x}, {chunk_z})')

    print(type(nbt_data))
    nbt_data['DataVersion'] = TAG(name='DataVersion', value=100)

    return cls(nbt_data)
anvil.Chunk.from_region = from_region_override

In [4]:
try:
    chunk = anvil.Chunk.from_region(anvil.Chunk, region, 0, 0)
    block = chunk.get_block(2, 66, 5)

    print(region)
    print(block)
    print(block.id)
    # print(block.properties) # 'OldBlock' object has no attribute 'properties'
    print(chunk.get_palette(15))
    nbt_for_chunk = region.chunk_data(0, 0)
    print(nbt_for_chunk)
except Exception as e:
    traceback.print_exc()

print(len(region.data))

<class 'nbt.nbt.NBTFile'>
OldBlock(id=0, data=0)
0
None
{TAG_Byte('Natura.Retrogen'): 1, TAG_Compound('Thaumcraft'): {1 Entries}, TAG_Compound('NaturaWorld'): {0 Entries}, TAG_Compound('Forestry'): {0 Entries}, TAG_Compound('Level'): {12 Entries}, TAG_Byte('gcewing.sg.oresGenerated'): 0, TAG_Compound('CoFHWorld'): {3 Entries}, TAG_Compound('RetrogenData_mrtjp_gen'): {2 Entries}}
12288


In [15]:
def getNonDunder(obj):
    return [f for f in dir(obj) if not f[:2] + f[-2:] == '____']

getNonDunder(nbt_for_chunk['Level'])

['_MutableMapping__marker',
 '_abc_impl',
 '_parse_buffer',
 '_render_buffer',
 'clear',
 'get',
 'id',
 'items',
 'iteritems',
 'keys',
 'name',
 'namestr',
 'pop',
 'popitem',
 'pretty_tree',
 'setdefault',
 'tag_info',
 'tags',
 'update',
 'value',
 'values',
 'valuestr']

In [6]:
nbt_for_chunk['Level'].keys()

['LightPopulated',
 'zPos',
 'HeightMap',
 'Sections',
 'LastUpdate',
 'V',
 'Biomes',
 'InhabitedTime',
 'xPos',
 'TerrainPopulated',
 'TileEntities',
 'Entities']

In [20]:
for k in nbt_for_chunk['Level'].keys():
    curr_nbt = nbt_for_chunk['Level'][k]
    print(k)
    print(curr_nbt.pretty_tree())
    print()

LightPopulated
TAG_Byte('LightPopulated'): 0

zPos
TAG_Int('zPos'): 0

HeightMap
TAG_Int_Array('HeightMap'): [256 int(s)]

Sections
TAG_List('Sections'): [4 TAG_Compound(s)]
{
	TAG_Compound: {7 Entries}
	{
		TAG_Byte_Array('Blocks'): [4096 byte(s)]
		TAG_Byte_Array('SkyLight'): [2048 byte(s)]
		TAG_Byte_Array('Blocks16'): [8192 byte(s)]
		TAG_Byte('Y'): 0
		TAG_Byte_Array('Data16'): [8192 byte(s)]
		TAG_Byte_Array('BlockLight'): [2048 byte(s)]
		TAG_Byte_Array('Data'): [2048 byte(s)]
	}
	TAG_Compound: {7 Entries}
	{
		TAG_Byte_Array('Blocks'): [4096 byte(s)]
		TAG_Byte_Array('SkyLight'): [2048 byte(s)]
		TAG_Byte_Array('Blocks16'): [8192 byte(s)]
		TAG_Byte('Y'): 1
		TAG_Byte_Array('Data16'): [8192 byte(s)]
		TAG_Byte_Array('BlockLight'): [2048 byte(s)]
		TAG_Byte_Array('Data'): [2048 byte(s)]
	}
	TAG_Compound: {7 Entries}
	{
		TAG_Byte_Array('Blocks'): [4096 byte(s)]
		TAG_Byte_Array('SkyLight'): [2048 byte(s)]
		TAG_Byte_Array('Blocks16'): [8192 byte(s)]
		TAG_Byte('Y'): 2
		TAG_Byte

In [38]:
for i in range(4):
    print(f'y={i}', Counter(nbt_for_chunk['Level']['Sections'][i]['Blocks']))

y=0 Counter({0: 3035, 121: 1061})
y=1 Counter({121: 4096})
y=2 Counter({121: 4096})
y=3 Counter({121: 3328, 0: 768})


In [48]:
ore_chunk_nbt = region.chunk_data(1, 1)
print(ore_chunk_nbt)
# ore_chunk_nbt['gcewing.sg.oresGenerated'].pretty_tree()

None
